In [ ]:
import subprocess
libraries_to_install = ['keras-tuner', 'ipynb']

for library in libraries_to_install:
    subprocess.run(["pip", "install", library], check=True)

In [ ]:
import pandas as pd
import numpy as np
import math

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
import logging; tf.get_logger().setLevel(logging.ERROR)
from datetime import date
from keras import Model, layers, models, optimizers, losses, metrics
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.losses import *
from keras.metrics import *
import time
import sys
import keras_tuner
from keras_tuner import Hyperband

In [ ]:
from pandas_datareader import data as pdr
from sklearn.preprocessing import StandardScaler
from scipy import stats
import pandas as pd
from datetime import date
import numpy as np
import random
tf.random.set_seed(42)
random.seed(42)
np.random.seed(42)
import glob, os
from joblib import dump, load
from ipynb.fs.full.Data_Preparation import getProcessedData
from ipynb.fs.full.Data_Preparation import getFilenames

In [ ]:
path = os.getcwd()

In [ ]:
# Define the list of currency tickers used in the analysis
fx_tic = ['USDEUR',
            'USDJPY',
            'USDGBP',
            'USDCHF',
            'USDNZD',
            'USDCAD',
            'USDSEK',
            'USDDKK',
            'USDNOK',
            'EURJPY',
            'EURGBP',
            'EURCHF',
            'EURNZD',
            'EURCAD',
            'EURSEK',
            'EURDKK',
            'EURNOK']

# Create a dictionary that maps each currency ticker to an index
fx_dic = {'USDEUR': 0,
 'USDJPY': 1,
 'USDGBP': 2,
 'USDCHF': 3,
 'USDNZD': 4,
 'USDCAD': 5,
 'USDSEK': 6,
 'USDDKK': 7,
 'USDNOK': 8,
 'EURJPY': 9,
 'EURGBP': 10,
 'EURCHF': 11,
 'EURNZD': 12,
 'EURCAD': 13,
 'EURSEK': 14,
 'EURDKK': 15,
 'EURNOK': 16}

In [ ]:
# Create a dictionary that defines the top 20 features for each currency ticker. These features are selected by SFFS
feat_20={
    'USDEUR': [0, 6, 11, 12, 16, 22, 23, 24, 25, 28, 30, 31, 33, 38, 44, 45, 48, 49, 54, 59],
    'USDJPY': [0, 13, 14, 15, 16, 18, 19, 20, 21, 22, 26, 28, 31, 33, 35, 38, 41, 47, 51, 53],
    'USDGBP': [1, 2, 5, 13, 16, 18, 23, 24, 25, 31, 35, 36, 38, 41, 42, 46, 48, 51, 52, 54],
    'USDCHF': [2, 5, 7, 8, 10, 11, 17, 25, 27, 35, 41, 43, 47, 48, 51, 52, 53, 54, 55, 57],
    'USDNZD': [5, 6, 8, 10, 13, 14, 17, 20, 21, 23, 24, 25, 33, 37, 40, 43, 51, 52, 53, 57],
    'USDCAD': [5, 8, 10, 11, 17, 18, 20, 23, 24, 25, 31, 33, 35, 40, 42, 52, 54, 55, 56, 57],
    'USDSEK': [7, 8, 10, 11, 17, 18, 21, 23, 24, 25, 26, 27, 31, 33, 35, 40, 42, 43, 53, 55],
    'USDDKK': [1, 3, 7, 13, 14, 17, 22, 23, 26, 27, 28, 29, 30, 31, 35, 44, 48, 51, 52, 53],
    'USDNOK': [0, 2, 4, 5, 7, 15, 17, 18, 20, 22, 23, 26, 27, 33, 34, 37, 47, 49, 51, 57],
    'EURJPY': [0, 1, 4, 9, 13, 20, 21, 23, 24, 25, 26, 29, 31, 41, 44, 47, 51, 53, 54, 56],
    'EURGBP': [4, 5, 7, 8, 13, 17, 18, 20, 21, 23, 24, 25, 27, 41, 42, 47, 51, 52, 54, 55],
    'EURCHF': [4, 11, 12, 14, 17, 18, 19, 26, 29, 32, 33, 34, 39, 40, 45, 47, 49, 50, 51, 52],
    'EURNZD': [8, 9, 10, 11, 16, 18, 21, 23, 26, 28, 31, 35, 36, 38, 46, 50, 51, 52, 53, 55],
    'EURCAD': [5, 8, 10, 11, 18, 20, 21, 23, 24, 25, 26, 28, 32, 33, 41, 42, 43, 46, 48, 58],
    'EURSEK': [2, 4, 8, 10, 11, 13, 15, 20, 22, 23, 33, 39, 40, 41, 44, 48, 51, 54, 55, 59],
    'EURDKK': [1, 6, 8, 10, 17, 21, 26, 30, 31, 35, 37, 43, 44, 45, 47, 48, 50, 51, 55, 58],
    'EURNOK': [2, 8, 10, 11, 13, 17, 18, 21, 23, 24, 25, 40, 41, 42, 43, 47, 53, 56, 57, 58]
}

In [ ]:
# Function to compute accuracy scores using an LSTM model for a list of ticker names
def LSTM_accuracy_scores(ticker_names,upload = False):

    # Initialize lists to store evaluation metrics
    accuracy_scores = []
    balanced_accuracy_scores = []
    f1_scores = []
    
    # Dictionaries to store models, predictions, and probabilities
    fitted_models = {}
    ticker_pred = {}
    ticker_prob = {}
    
    # Loop over each ticker in the list
    for i in range(len(ticker_names)):
        # Data split
        X_train, X_val1, X_val2, X_test, y_train, y_val1, y_val2, y_test = getProcessedData(getFilenames(ticker_names)[i])

        # For each market, we only use top 20 features selected by SFFS.
        X_train_LSTM = X_train.iloc[:,feat_20[ticker_names[i]]].copy()
        X_val_LSTM = X_val1.iloc[:,feat_20[ticker_names[i]]].copy()
        X_test_LSTM = X_val2.iloc[:,feat_20[ticker_names[i]]].copy()
        y_train_LSTM = y_train.copy()
        y_val_LSTM = y_val1.copy()
        y_test_LSTM = y_val2.copy()
        cpst = 'compare'+ticker_names[i]

        # Check if the model should be loaded directly
        if upload == True:
        # For ensure that the model does not change when it is used again and is easy to reproduce.
            best_model = tf.keras.models.load_model(path + '/LSTM_for_FX/LSTM_' + ticker_names[i] + '_CPST_66.h5')
        
        # Otherwise, train the model from scratch
        elif upload == False:
            # Train a new model and get the best one
            best_model = LSTM_fit(X_train_LSTM, X_val_LSTM, y_train_LSTM, y_val_LSTM, cpst)
            # Save the trained model as a h5 format file
            best_model.save(path + '/LSTM_for_FX/LSTM_' + ticker_names[i] + '_CPST_66.h5')
        else:
            print('upload should be True or False')

        # Visualize the model's architecture
        tf.keras.utils.plot_model(
            best_model,
            to_file = path + '/LSTM_for_FX/LSTM_' + ticker_names[i] + '_CPST_66.png',
        )
        
        # Predict the test set results
        y_pred, test_predict = LSTM_predict(best_model, X_test_LSTM)

        # Store results in dictionaries
        fitted_models[ticker_names[i]] = best_model
        ticker_pred[ticker_names[i]] = y_pred
        ticker_prob[ticker_names[i]] = test_predict

        # Compute and store evaluation metrics
        # Add accuracy scores
        accuracy_scores.append(accuracy_score(y_test_LSTM,y_pred))
        # Add balanced accuracy scores
        balanced_accuracy_scores.append(balanced_accuracy_score(y_test_LSTM,y_pred))
        # Add F1 scores
        f1_scores.append(f1_score(y_test_LSTM,y_pred))

    # Create dataframes for accuracy and F1 scores
    df_accuracy = pd.DataFrame(list(zip(ticker_names, accuracy_scores)),columns=['Market','Accuracy'])
    df_f1 = pd.DataFrame(list(zip(ticker_names, f1_scores)),columns=['Market','F1 Score'])
    # Add balanced accuracy scores to accuracy dataframe
    df_accuracy['balanced'] = balanced_accuracy_scores

    return df_accuracy, df_f1, fitted_models, ticker_pred, ticker_prob

In [ ]:
# Function to build the LSTM model with hyperparameters
def model_build(hp):

    # hp: A hyperparameter instance (from Keras Tuner) that contains values for hyperparameters like the number 
    #     of LSTM layers, units in each LSTM layer, activations, dropout rates, and learning rate.

    # Define the input shape. The shape is (20, 1)
    input = tf.keras.Input(shape=(20,1))
    x = input

    # Dynamically add LSTM layers based on hyperparameters.
    # The loop will iterate based on the number of LSTM layers specified in hyperparameters.
    for i in range(hp.Int('num_lstm_layers', 1, 3, default = 2)):
        x = layers.LSTM(# 'units' determines the dimensionality of the output space
                        units = hp.Int('units_'+str(i), min_value = 32, max_value = 256, step = 32),
                        # 'activation' is the activation function to use
                        activation = hp.Choice('activation_'+str(i), values = ['tanh', 'relu']),
                        # 'return_sequences' is True for all but the last LSTM layer to ensure each LSTM returns sequences to the next one
                        return_sequences = True if i < hp.Int('num_lstm_layers', 1, 3, default = 2)-1 else False)(x)
        
        # Add a dropout layer after the first LSTM layer to prevent overfitting
        # Dropout randomly sets a fraction of the input units to 0 at each update during training, which helps to prevent overfitting
        if i == 0:
            x = layers.Dropout(rate=hp.Float('dropout_rate_'+str(i), 0.3, 0.6, step=0.1))(x)

    # Add output layer
    # Dense layer with a sigmoid activation function is used for binary classification tasks
    x = layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.models.Model(inputs=input, outputs=x)

    # Compile the model with optimizer, loss, and metrics
    model.compile(
        # The optimizer is Adam, which is a popular choice for training deep learning models
        optimizer=tf.keras.optimizers.Adam(
            # Learning rate hyperparameter with a logarithmic scale
            hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling='log')
        ),
        # 'binary_crossentropy' is used as the loss function for binary classification tasks
        loss = 'binary_crossentropy',
        # Measure the accuracy of the model during training and testing
        metrics = ['accuracy']
    )

    return model


In [ ]:
# Function to train the LSTM model
def LSTM_fit(X_train_LSTM, X_val_LSTM, y_train_LSTM, y_val_LSTM, cpst):

    # cpst: project name for the Hyperband tuner directory
    
    # Callback to reduce learning rate when 'val_loss' (validation loss) has stopped improving
    # Reducing learning rate can help the model converge to a local minimum more precisely
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-4)

    # Setting up the Hyperband tuner
    # Hyperband is an optimized version of random search which uses early stopping to speed up hyperparameter tuning
    tuner = Hyperband(
        # Model-building function defined previously
        model_build,
        # Optimization objective
        objective='val_accuracy',
        # Maximum number of epochs to train a single model
        max_epochs = 50,
        # Directory to save tuning logs and trained models
        directory='/content/gdrive/MyDrive/Capstone-main/LSTM_for_FX',
        # Name of the project, helps in organizing different tuning sessions
        project_name=cpst,
        # Random seed for reproducibility
        seed = 42
    )
    
    # Early stopping callback to stop training when 'val_accuracy' (validation accuracy) stops improving
    # Helps prevent overfitting and reduce unnecessary training epochs
    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience = 5)
    # Perform hyperparameter search
    history = tuner.search(
        # Training features
        X_train_LSTM,
        # Training labels
        y_train_LSTM,
        # Maximum number of epochs per model
        epochs=50,
        # Validation data
        # We use pre-divided validation set due to LSTM is a model for processing time series type data
        validation_data = (X_val_LSTM, y_val_LSTM),
        # Not shuffling as it's time series data
        shuffle = False,
        # Number of samples per gradient update
        batch_size=32,
        # List of callbacks to apply during training
        callbacks=[stop_early,reduce_lr]
    )

    # Get the optimal hyperparameters and build the model using them
    best_hp = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hp)

    return best_model

In [ ]:
# Function to predict using the LSTM model
def LSTM_predict(best_model, X_test):
    test_predict = best_model.predict(X_test)

    pred = []

    for j in test_predict:
        # Convert probabilities into class labels using 0.5 as threshold
        if(j>0.5): 
            pred.append(1)
        else:
            pred.append(0)

    return pred, test_predict

In [ ]:
# Execute the LSTM_accuracy_scores function
df_acc, df_f1, fitted_models, ticker_pred, ticker_prob = LSTM_accuracy_scores(fx_tic)

Trial 90 Complete [00h 00m 16s]
val_accuracy: 0.5300127863883972

Best val_accuracy So Far: 0.5657726526260376
Total elapsed time: 00h 18m 38s
25/25 [==============================] - 0s 2ms/step


In [ ]:
df_acc

,Market,Accuracy,balanced
0,USDEUR,0.516667,0.500000
1,USDJPY,0.478205,0.475812
2,USDGBP,0.514103,0.515022
3,USDCHF,0.508974,0.511780
4,USDNZD,0.515385,0.521291
5,USDCAD,0.487179,0.487152
6,USDSEK,0.503846,0.500000
7,USDDKK,0.507692,0.503106
8,USDNOK,0.487179,0.487523
9,EURJPY,0.484615,0.500000


In [ ]:
df_f1

,Market,F1 Score
0,USDEUR,0.681319
1,USDJPY,0.435506
2,USDGBP,0.515964
3,USDCHF,0.515803
4,USDNZD,0.487805
5,USDCAD,0.483204
6,USDSEK,0.670077
7,USDDKK,0.452991
8,USDNOK,0.497487
9,EURJPY,0.000000


In [ ]:
df_acc.to_csv(path + '/LSTM_acc_df_val1_for_vali.csv', index=False)